## Proximity Matrix

import library yang dibutuhkan

In [1]:
import pandas as pd
import numpy as np
import math

membaca excel menggunakan library pandas

In [2]:
exc = pd.read_excel('DataCoba.xlsx', sheet_name='Sheet1')
# slicing seluruh data dari semua atribut kecuali class atau label
data = exc.iloc[:,1:]
data

,Feat 1,Feat 2,Feat 3
0,15,0,5
1,0,20,0
2,6,3,5
3,2,7,0
4,1,0,3
5,0,1,1
6,8,9,3
7,5,1,7


Merubah dataframe menjadi numpy agar bisa melakukan perhitungan eucledian distance dengan menggunakan library numpy

In [3]:
data = data.to_numpy()

Menghitung eucledian distance dari setiap data

In [4]:
prxy_mtrx = []
for poin1 in data:
  row = []
  for poin2 in data:
    # np linalg norm adalah library dari numpy untuk menghitung eucledian distance dari 2 poin
    row.append(np.linalg.norm(poin1-poin2))
  prxy_mtrx.append(row)
  
# penamaan index dataframe
n_data =[]
for i in range(len(prxy_mtrx)):
  n_data.append(f'Data {i+1}')

df_proximity_mtrx = pd.DataFrame(prxy_mtrx, columns=n_data, index=n_data)

Menampilkan hasil proximity matrix dalam bentuk dataframe

In [5]:
df_proximity_mtrx

,Data 1,Data 2,Data 3,Data 4,Data 5,Data 6,Data 7,Data 8
Data 1,0.000000,25.495098,9.486833,15.588457,14.142136,15.556349,11.575837,10.246951
Data 2,25.495098,0.000000,18.708287,13.152946,20.248457,19.026298,13.928388,20.856654
Data 3,9.486833,18.708287,0.000000,7.549834,6.164414,7.483315,6.633250,3.000000
Data 4,15.588457,13.152946,7.549834,0.000000,7.681146,6.403124,7.000000,9.695360
Data 5,14.142136,20.248457,6.164414,7.681146,0.000000,2.449490,11.401754,5.744563
Data 6,15.556349,19.026298,7.483315,6.403124,2.449490,0.000000,11.489125,7.810250
Data 7,11.575837,13.928388,6.633250,7.000000,11.401754,11.489125,0.000000,9.433981
Data 8,10.246951,20.856654,3.000000,9.695360,5.744563,7.810250,9.433981,0.000000


## Gini and Entropy

Membaca dataset excel menggunakan library pandas untuk merubah menjadi dataframe

In [6]:
data = pd.read_excel('DataCoba.xlsx', sheet_name='Sheet2')
data.head()

,Feat 1,Feat 2,Feat 3,Feat 4,Class
0,Jeruk,TIDAK ADA,L,LOKAL,C0
1,Jeruk,ADA,M,LOKAL,C0
2,Mangga,TIDAK ADA,S,IMPORT,C0
3,Mangga,TIDAK ADA,M,IMPORT,C0
4,Jeruk,ADA,L,IMPORT,C0


mencari total data dan mencari jumlah masing-masing class atau label 

In [7]:
total_data = data.shape[0]
n_class = data['Class'].value_counts()
print(n_class)

C0    25
C1    25
Name: Class, dtype: int64


Menghitung Gini dan Entropy dari parent

In [8]:
n_class_dict = n_class.to_dict()
gini_sum = 0
entropy_parent = 0
for cls,val in n_class_dict.items():
  gini_sum = gini_sum + (val/total_data)**2
  entropy_parent = entropy_parent - (val/total_data * math.log2(val/total_data)) 
gini_parent = 1-gini_sum

temp_dict = {}
for cls,val in n_class_dict.items():
  temp_dict[cls] = [val]
temp_dict['Gini'] = [gini_parent]
temp_dict['Entropy'] = [entropy_parent]

parent = pd.DataFrame(temp_dict, index=['Parent'])
parent

,C0,C1,Gini,Entropy
Parent,25,25,0.5,1.0


Menghitung Jumlah Item Dari Masing masing feature

In [9]:
# mengambil semua data pada masing" atribut kecuali class
features = data.loc[:, data.columns != 'Class']
dict_features_val_counts = {}
for i in features:
  dict_features_val_counts[i] = features[i].value_counts().to_dict()


for feat,val in dict_features_val_counts.items():
  print(feat, val)

Feat 1 {'Mangga': 20, 'Jeruk': 19, 'Apel': 11}
Feat 2 {'ADA': 31, 'TIDAK ADA': 19}
Feat 3 {'S': 22, 'M': 16, 'L': 12}
Feat 4 {'LOKAL': 25, 'IMPORT': 25}


Menghitung Gini dan Entropy dari masing masing item dan menghitung gain gini dan gain entropy dari masing masing feature

In [10]:
temp_dict_feat = {}
for feat,val in dict_features_val_counts.items():
  gain_gini = 0
  for key,item in val.items():
    gini_sum = 0
    entropy = 0
    for cls in n_class_dict:
      new_df = data[((data[feat] == key) & (data['Class'] == cls))]
      # jumlah item tertentu pada class tertentu
      count = new_df[feat].count()
      # jumlah item tertentu pada semua class
      total = dict_features_val_counts[feat][key]
      # probability
      probability = count/total
      # gini sum
      gini_sum = gini_sum + (probability)**2
      # entropy
      try:
        entropy = entropy - probability * math.log2(probability)
      except:
        # jika perhitungan log error default nilai entropy 0
        entropy = 0
      if feat not in temp_dict_feat:
        temp_dict_feat[feat] = { key : { cls : count} }
      else :
        if key not in temp_dict_feat[feat]:
          temp_dict_feat[feat][key] = { cls : count}
        else:
          temp_dict_feat[feat][key][cls] = count
      # print(feat,key,cls, count)
    # total item
    temp_dict_feat[feat][key]['Total'] =  total
    # rumus gini
    gini = 1-gini_sum
    temp_dict_feat[feat][key]['Gini'] = gini
    temp_dict_feat[feat][key]['Entropy'] = entropy


for feat,val in temp_dict_feat.items():
  _var_gini = 0
  _var_entropy = 0
  for item,vl in val.items():
    # variable penampung untuk perhitungan gain gini
    _var_gini = _var_gini + (vl['Gini'] * (vl['Total']/total_data))
    # variable penampung untuk perhitungan gain entropy
    _var_entropy = _var_entropy + (vl['Entropy'] * (vl['Total']/total_data))
  gain_gini = gini_parent - _var_gini
  gain_entropy = entropy_parent - _var_entropy
  temp_dict_feat[feat]['Gain Gini'] = gain_gini
  temp_dict_feat[feat]['Gain Entropy'] = gain_entropy

for feat,val in temp_dict_feat.items():
  print(feat, val)

Feat 1 {'Mangga': {'C0': 10, 'C1': 10, 'Total': 20, 'Gini': 0.5, 'Entropy': 1.0}, 'Jeruk': {'C0': 12, 'C1': 7, 'Total': 19, 'Gini': 0.46537396121883656, 'Entropy': 0.9494520153879484}, 'Apel': {'C0': 3, 'C1': 8, 'Total': 11, 'Gini': 0.39669421487603307, 'Entropy': 0.8453509366224365}, 'Gain Gini': 0.03588516746411485, 'Gain Entropy': 0.05323102809564362}
Feat 2 {'ADA': {'C0': 17, 'C1': 14, 'Total': 31, 'Gini': 0.49531737773152973, 'Entropy': 0.9932338197397066}, 'TIDAK ADA': {'C0': 8, 'C1': 11, 'Total': 19, 'Gini': 0.48753462603878117, 'Entropy': 0.9819407868640976}, 'Gain Gini': 0.007640067911714732, 'Gain Entropy': 0.011057532753024901}
Feat 3 {'S': {'C0': 14, 'C1': 8, 'Total': 22, 'Gini': 0.4628099173553719, 'Entropy': 0.9456603046006401}, 'M': {'C0': 6, 'C1': 10, 'Total': 16, 'Gini': 0.46875, 'Entropy': 0.9544340029249649}, 'L': {'C0': 5, 'C1': 7, 'Total': 12, 'Gini': 0.48611111111111105, 'Entropy': 0.9798687566511528}, 'Gain Gini': 0.02969696969696972, 'Gain Entropy': 0.0433220834

Menampilkan gain gini dan gain entropy dari masing-masing feature kemudian mencari gain gini dan gain entropy terbesar

In [11]:
gain_gini_terbesar = ['',0]
gain_entropy_terbesar = ['',0]
for feat in temp_dict_feat:
  if temp_dict_feat[feat]["Gain Gini"] > gain_gini_terbesar[1]:
    gain_gini_terbesar[0] = feat
    gain_gini_terbesar[1] = temp_dict_feat[feat]["Gain Gini"]
  if temp_dict_feat[feat]["Gain Entropy"] > gain_entropy_terbesar[1]:
    gain_entropy_terbesar[0] = feat
    gain_entropy_terbesar[1] = temp_dict_feat[feat]["Gain Entropy"]
  print(f'{feat} memiliki Gain Gini : {temp_dict_feat[feat]["Gain Gini"]}')
  print(f'{feat} memiliki Gain Entropy : {temp_dict_feat[feat]["Gain Entropy"]}')

Feat 1 memiliki Gain Gini : 0.03588516746411485
Feat 1 memiliki Gain Entropy : 0.05323102809564362
Feat 2 memiliki Gain Gini : 0.007640067911714732
Feat 2 memiliki Gain Entropy : 0.011057532753024901
Feat 3 memiliki Gain Gini : 0.02969696969696972
Feat 3 memiliki Gain Entropy : 0.0433220834434529
Feat 4 memiliki Gain Gini : 0.007200000000000095
Feat 4 memiliki Gain Entropy : 0.010412478777944378


In [12]:
print(f'Best split berdasarkan gain gini terbesar yaitu atribut {gain_gini_terbesar[0]} sebesar {gain_gini_terbesar[1]}')
print(f'Best split berdasarkan gain entropy terbesar yaitu atribut {gain_entropy_terbesar[0]} sebesar {gain_entropy_terbesar[1]}')

Best split berdasarkan gain gini terbesar yaitu atribut Feat 1 sebesar 0.03588516746411485
Best split berdasarkan gain entropy terbesar yaitu atribut Feat 1 sebesar 0.05323102809564362
